In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, DecimalType, IntegerType,FloatType
import pandas as pd 
# Build SparkSession
#spark = SparkSession.builder.master("local[*]").appName("ETL Pipeline").getOrCreate()

In [2]:
master = "spark://192.53.165.168:7077"
master = "local"
#        .master(master)\

# SparkSession
#                 .builder
#                 .appName('example-pyspark-read-and-write-from-hive')
#                 .config("hive.metastore.uris", "thrift://localhost:9083", conf=SparkConf())
#                 .enableHiveSupport()
#                 .getOrCreate()

spark = SparkSession.builder\
        .appName('taxis')\
        .master(master)\
        .config("hive.metastore.uris", "thrift://192.53.165.168:9083", conf=SparkConf())\
        .enableHiveSupport()\
        .getOrCreate()
# Set Logging Level to WARN
spark.sparkContext.setLogLevel("WARN")


22/07/13 10:28:32 WARN Utils: Your hostname, LAPTOP-GOF4EGS4 resolves to a loopback address: 127.0.1.1; using 172.17.204.25 instead (on interface eth0)
22/07/13 10:28:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/13 10:28:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark.read.csv("hdfs://")

In [4]:
spark.getActiveSession()

In [4]:
df_taxis=spark.read.option("header", "true").parquet("../Data/Taxis.parquet")
df_taxis.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+--------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Tiempo_Viaje_s|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+--------------+
|       1| 2017-12-31 19:21:05|  2017-12-31 19:24:23|              1|          0.5|         1|                 N|          41|          24|           2|        4.5|  0.5|    0.5|

In [5]:
Vendor_columns=["Id_Vendor","Vendor_Name"]
Vendor=[(1,"Creative Mobiles Technologies"),(2,"VeriFone Inc.")]
df_vendor=spark.createDataFrame(Vendor,Vendor_columns)
df_vendor.show()

+---------+--------------------+
|Id_Vendor|         Vendor_Name|
+---------+--------------------+
|        1|Creative Mobiles ...|
|        2|       VeriFone Inc.|
+---------+--------------------+



In [6]:
Rate_columns=['Id_Rate', 'Rate_Name']
Rate=[
    (1,"Standard rate"),
    (2,"JFK"),
    (3,"Newark"),
    (4,"Nassau or Westchester"),
    (5,"Negotiated fare"),
    (6,"Group ride")]

df_rate=spark.createDataFrame(Rate,Rate_columns)
df_rate.show()

+-------+--------------------+
|Id_Rate|           Rate_Name|
+-------+--------------------+
|      1|       Standard rate|
|      2|                 JFK|
|      3|              Newark|
|      4|Nassau or Westche...|
|      5|     Negotiated fare|
|      6|          Group ride|
+-------+--------------------+



In [7]:
Payment_columns=['Id_Payment', 'Payment_type']
Payment= [
    (1,"Credit card"),
    (2,"Cash"),
    (3,"No charge"),
    (4,"Dispute"),
    (5,"Unknown"),
    (6,"Voided trip")]
df_payment=spark.createDataFrame(Payment,Payment_columns)
df_payment.show()

+----------+------------+
|Id_Payment|Payment_type|
+----------+------------+
|         1| Credit card|
|         2|        Cash|
|         3|   No charge|
|         4|     Dispute|
|         5|     Unknown|
|         6| Voided trip|
+----------+------------+



In [8]:
Borough_columns=["ID_Borough","Borough","Latitud","Longitud"]
Borough = [
    (1,"Manhattan",40.776676,-73.971321),
    (2,"Brooklyn",40.650002,-73.949997),
    (3,"Bronx",40.837048,-73.865433),
    (4,"Queens",40.742054,-73.769417),
    (5,"Staten Island",40.579021,-74.151535),
]
Borough_schema = StructType([ 
    StructField("ID_Borough",IntegerType(),True), 
    StructField("Borough",StringType(),True), 
    StructField("Latitud",FloatType(),True), 
    StructField("Longitud", FloatType(), True), 
])
df_borough=spark.createDataFrame(data=Borough,schema=Borough_schema).cache()
df_borough.show()

+----------+-------------+---------+----------+
|ID_Borough|      Borough|  Latitud|  Longitud|
+----------+-------------+---------+----------+
|         1|    Manhattan|40.776676| -73.97132|
|         2|     Brooklyn|    40.65|    -73.95|
|         3|        Bronx|40.837048| -73.86543|
|         4|       Queens|40.742054| -73.76942|
|         5|Staten Island| 40.57902|-74.151535|
+----------+-------------+---------+----------+



In [23]:
location_schema = StructType([ 
    StructField("LocationID",IntegerType(),True), 
    StructField("Borough",StringType(),True), 
    StructField("Zone",StringType(),True), 
    StructField("service_zone", StringType(), True), 
])
data_location=spark.read.option("header", "true").csv("../taxi+_zone_lookup.csv",schema=location_schema)
data_location.show(5)

NameError: name 'spark' is not defined

In [10]:
data_location = data_location.join(df_borough,on='Borough',how='left').drop("Borough", "Latitud", "Longitud")
data_location=data_location.dropna()[["LocationID", "ID_Borough", "Zone", "service_zone"]]
data_location.show(5)

+----------+----------+--------------------+------------+
|LocationID|ID_Borough|                Zone|service_zone|
+----------+----------+--------------------+------------+
|         2|         4|         Jamaica Bay|   Boro Zone|
|         3|         3|Allerton/Pelham G...|   Boro Zone|
|         4|         1|       Alphabet City| Yellow Zone|
|         5|         5|       Arden Heights|   Boro Zone|
|         6|         5|Arrochar/Fort Wad...|   Boro Zone|
+----------+----------+--------------------+------------+
only showing top 5 rows



In [21]:
def tabla_calendario(start='2018-01-01', end='2018-12-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["Year"] = df.Date.dt.year
    df["Month"] = df.Date.dt.month
    df["Day"] = df.Date.dt.day
    df["Week"] = df.Date.dt.isocalendar().week
    df["Quarter"] = df.Date.dt.quarter
    df["Year_half"] = (df.Quarter + 1) // 2
    df["Weekday"] = df.Date.dt.day_of_week
    return df
# calendario=spark.createDataFrame(tabla_calendario(start='2018-01-01', end='2018-12-31'))
# calendario.show()

In [22]:
df_calendario=tabla_calendario(start='2018-01-01', end='2018-12-31')

In [6]:
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError
DATABASE_LOCATION='hive://192.53.165.168:10000/TaxisDB'
# DATABASE_LOCATION="apachehive:///?Server=127.0.0.1&;Port=10000&TransportMode=BINARY"
engine = sqlalchemy.create_engine(DATABASE_LOCATION,echo=True)
cursor = engine.connect()

In [9]:
cursor.execute("create database taxisdb")

2022-07-13 11:26:34,633 INFO sqlalchemy.engine.Engine create database taxisdb
2022-07-13 11:26:34,641 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-13 11:26:36,139 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError
DATABASE_LOCATION='hive://192.53.165.168:10000/taxisdb'
# DATABASE_LOCATION="apachehive:///?Server=127.0.0.1&;Port=10000&TransportMode=BINARY"
engine = sqlalchemy.create_engine(DATABASE_LOCATION,echo=True)
cursor = engine.connect()

In [30]:
cursor.execute("show databases").all()

2022-07-13 14:00:12,351 INFO sqlalchemy.engine.Engine show databases
2022-07-13 14:00:12,354 INFO sqlalchemy.engine.Engine [raw sql] {}


[('default',), ('taxisdb',)]

In [31]:
query="""
create table if not exists location(
    LocationID int,
    Borough string,
    Zone string,
    service_zone string,
    primary key(LocationID) disable novalidate 
)
"""
cursor.execute(query)

2022-07-13 14:00:15,299 INFO sqlalchemy.engine.Engine 
create table if not exists location(
    LocationID int,
    Borough string,
    Zone string,
    service_zone string,
    primary key(LocationID) disable novalidate 
)

2022-07-13 14:00:15,301 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-13 14:00:15,619 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
df_location=pd.read_csv("../taxi+_zone_lookup.csv")
df_location

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [33]:
# try:
df_location.to_sql("location", engine, index=False, if_exists='append')

# except Exception as e:  #Want to capture all errors
#     print("Shiat broke down ...")
#     print(e)

2022-07-13 14:00:42,956 INFO sqlalchemy.engine.Engine DESCRIBE location
2022-07-13 14:00:42,959 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-13 14:00:43,627 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-13 14:00:43,639 INFO sqlalchemy.engine.Engine INSERT INTO TABLE `location` VALUES (%(LocationID)s, %(Borough)s, %(Zone)s, %(service_zone)s)
2022-07-13 14:00:43,641 INFO sqlalchemy.engine.Engine [dialect b'hive'+b'thrift' does not support caching 0.00401s] ({'LocationID': 1, 'Borough': 'EWR', 'Zone': 'Newark Airport', 'service_zone': 'EWR'}, {'LocationID': 2, 'Borough': 'Queens', 'Zone': 'Jamaica Bay', 'service_zone': 'Boro Zone'}, {'LocationID': 3, 'Borough': 'Bronx', 'Zone': 'Allerton/Pelham Gardens', 'service_zone': 'Boro Zone'}, {'LocationID': 4, 'Borough': 'Manhattan', 'Zone': 'Alphabet City', 'service_zone': 'Yellow Zone'}, {'LocationID': 5, 'Borough': 'Staten Island', 'Zone': 'Arden Heights', 'service_zone': 'Boro Zone'}, {'LocationID': 6, 'Borough': 'Staten Island'

/home/mangoru/.local/lib/python3.8/site-packages/pandas/io/sql.py:857: SAWarning: Dialect b'hive':b'thrift' will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  result = conn.execute(self.table.insert(), data)


2022-07-13 14:00:46,053 INFO sqlalchemy.engine.Engine ROLLBACK


ProgrammingError: (pyhive.exc.ProgrammingError) No result set
[SQL: INSERT INTO TABLE `location` VALUES (%(LocationID)s, %(Borough)s, %(Zone)s, %(service_zone)s)]
[parameters: ({'LocationID': 1, 'Borough': 'EWR', 'Zone': 'Newark Airport', 'service_zone': 'EWR'}, {'LocationID': 2, 'Borough': 'Queens', 'Zone': 'Jamaica Bay', 'service_zone': 'Boro Zone'}, {'LocationID': 3, 'Borough': 'Bronx', 'Zone': 'Allerton/Pelham Gardens', 'service_zone': 'Boro Zone'}, {'LocationID': 4, 'Borough': 'Manhattan', 'Zone': 'Alphabet City', 'service_zone': 'Yellow Zone'}, {'LocationID': 5, 'Borough': 'Staten Island', 'Zone': 'Arden Heights', 'service_zone': 'Boro Zone'}, {'LocationID': 6, 'Borough': 'Staten Island', 'Zone': 'Arrochar/Fort Wadsworth', 'service_zone': 'Boro Zone'}, {'LocationID': 7, 'Borough': 'Queens', 'Zone': 'Astoria', 'service_zone': 'Boro Zone'}, {'LocationID': 8, 'Borough': 'Queens', 'Zone': 'Astoria Park', 'service_zone': 'Boro Zone'}  ... displaying 10 of 265 total bound parameter set

In [12]:
# def load_to_hive(): 
#         # load the .csv file data to spark dataframe
#         weather = spark.read \
#                     .format("parquet") \
#                     .option("header", "true") \
#                     .load("../Data/Taxis.parquet")
#         # print a dataframe to the console
#         weather.show()
 
#         ####Method 1 - write method ######
#         # save dataframe to a Hive table
#         weather.write \
#         .format("orc") \
#         .mode("overwrite") \
#         .saveAsTable("TaxisDB.weather")
 
# #Main program starts here
# if __name__ == "__main__":
#         appname = "ETL pipeline"
#         master = "local"
#         spark = SparkSession.builder.appName(appname).master(master).enableHiveSupport().getOrCreate()
#         load_to_hive()
#         spark.stop()

In [3]:
spark.sql("show databases").show()

22/07/12 21:05:26 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/07/12 21:05:26 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/07/12 21:05:33 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/07/12 21:05:33 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore mangoru@127.0.1.1
22/07/12 21:05:33 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
+---------+
|namespace|
+---------+
|  default|
+---------+

